In [23]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width: 100% !important; }</style>"))

# Chapter6. Working with Different Types of Data

* Booleans
* Numbers
* Strings
* Dates and timestamps
* Handling null
* Complex types
* User-defined functions


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [3]:
df = spark.read.format('csv') \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .load("data/retail-data/by-day/2010-12-01.csv")

df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



## Converting to Spark Types
* Converting native types to Spark types.
* lit function: converts a type in another language to its corresponding Spark representation.



In [4]:
from pyspark.sql.functions import lit
df.select(lit(5), lit("five"), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

## Working with Booleans
* for filtering
* Boolean statements consist of `and`, `or`, `true` and `false` 
* Spark will flatten all of chaining filters into one statement \
and perform the filter at the same time.

In [7]:
from pyspark.sql.functions import col

df.where(col("InvoiceNo") != 536365) \
  .select("InvoiceNo", "Description") \
  .show(5, False)

# other ways of expressing "does no equal"
df.where("InvoiceNo != 536365") \
  .show(5, False)
df.where("InvoiceNo <> 536365") \
  .show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows

+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                  |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|536366   |22633    |HAND WARMER UNION JACK       |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536366   |22632    |HAND WARMER RED POLKA DOT    |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536367   |8

In [12]:
from pyspark.sql.functions import instr, expr

priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()

# You can make columns by Boolean experssions and filter a DataFrame 
DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter)) \
  .where("isExpensive") \
  .select("UnitPrice", "isExpensive").show(5)

# You can not use reserved filters and just use experssions to filter
df.withColumn("isExpensive", expr("NOT UnitPrice <= 250")) \
  .where("isExpensive") \
  .select("Description", "UnitPrice").show(5)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+

+---------+-----------+
|UnitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



## Working with Numbers

* numeric column 간의 *, +, - 가 가능하다.
* power, round, bround etc 제공
* correlation coeffcient 와 같은 stat도 제공 

In [19]:
from pyspark.sql.functions import pow, round, bround

fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(2)

df.selectExpr(
    "CustomerId",
    "(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity").show(2)

df.select(round(lit("2.2")), bround(lit("2.6"))).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows

+-------------+--------------+
|round(2.2, 0)|bround(2.6, 0)|
+-------------+--------------+
|          2.0|           3.0|
|          2.0|           3.0|
+-------------+--------------+
only showing top 2 rows



In [21]:
from pyspark.sql.functions import corr

df.select(corr("Quantity", "UnitPrice")).show()
df.describe().show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+

+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|         UnitPrice|        CustomerID|       Country|
+-------+-----------------+------------------+--------------------+------------------+------------------+------------------+--------------+
|  count|             3108|              3108|                3098|              3108|              3108|              1968|          3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128| 4.151946589446603|15661.388719512195|          null|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|15.638659854603892|1854.4496996893627|          null|
|    min|          

In [29]:
colName = "UnitPrice"
quantileProbs = [0.5]
relError = 0.05

df.stat.approxQuantile("UnitPrice", quantileProbs, relError)
df.stat.crosstab("StockCode", "Quantity").show()
df.stat.freqItems(["StockCode", "Quantity"]).show()

+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7|  1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192|  2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288|  3| 30| 32| 33| 34| 36|384|  4| 40|432| 47| 48|480|  5| 50| 56|  6| 60|600| 64|  7| 70| 72|  8| 80|  9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|             22578|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0| 

In [33]:
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id(), "*").show()

+-----------------------------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|monotonically_increasing_id()|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+-----------------------------+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|                            0|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|                            1|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|                            2|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|                            3|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|                            4|   

# Working with Strings

In [36]:
from pyspark.sql.functions import initcap, lower, upper
df.select(col("Description"), 
          initcap(col("Description")), 
          lower(col("Description")),
          upper(col("Description"))).show(2)

+--------------------+--------------------+--------------------+--------------------+
|         Description|initcap(Description)|  lower(Description)|  upper(Description)|
+--------------------+--------------------+--------------------+--------------------+
|WHITE HANGING HEA...|White Hanging Hea...|white hanging hea...|WHITE HANGING HEA...|
| WHITE METAL LANTERN| White Metal Lantern| white metal lantern| WHITE METAL LANTERN|
+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [39]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim

df.select(
    ltrim(lit(" HELLO ")).alias("ltrim"),
    rtrim(lit(" HELLO ")).alias("rtrim"),
    trim(lit(" HELLO ")).alias("trim"),
    lpad(lit("HELLO"), 10, " ").alias("lp"),
    rpad(lit("HELLO"), 10, " ").alias("rp")).show(2)

+------+------+-----+----------+----------+
| ltrim| rtrim| trim|        lp|        rp|
+------+------+-----+----------+----------+
|HELLO | HELLO|HELLO|     HELLO|HELLO     |
|HELLO | HELLO|HELLO|     HELLO|HELLO     |
+------+------+-----+----------+----------+
only showing top 2 rows



### Regular Expressions
* regexp_extract(col, regex_string, number) 
* regexp_replace(col, regex_string, replace_str)
* translate(col, old, new)
* instr
* locate

In [44]:
from pyspark.sql.functions import regexp_replace, translate

regex_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(
    regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
    col("Description")).show(2)

# without regex
# translate: character 단위로 치환
df.select(translate(col("Description"), "LEET", "1337"), col("Description")) \
  .show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows

+----------------------------------+--------------------+
|translate(Description, LEET, 1337)|         Description|
+----------------------------------+--------------------+
|              WHI73 HANGING H3A...|WHITE HANGING HEA...|
|               WHI73 M37A1 1AN73RN| WHITE METAL LANTERN|
+----------------------------------+--------------------+
only showing top 2 rows



In [46]:
from pyspark.sql.functions import regexp_extract

extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(
    regexp_extract(col("Description"), extract_str, 1).alias("color_clean"),
    col("Description")).show(2)

from pyspark.sql.functions import instr
containsBlack = instr(col("Description"), "BLACK") >= 1
containsWhite = instr(col("Description"), "WHITE") >= 1
df.withColumn("hasSimpleColor", containsBlack | containsWhite)\
.where("hasSimpleColor")\
.select("Description").show(3, False)

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
+-----------+--------------------+
only showing top 2 rows

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



In [49]:
from pyspark.sql.functions import expr, locate

simpleColors = ["black", "white", "red", "green", "blue"]
def color_locator(column, color_string):
    return locate(color_string.upper(), column)\
        .cast("boolean")\
        .alias("is_" + color_string)

selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(expr("*")) # has to a be Column type
df.select(*selectedColumns).where(expr("is_white OR is_red"))\
    .select("Description").show(3, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



## Working with Dates and Timestamp

## 사용자 정의 함수
* 레코드 별로 처리하는 함수로, \
입출력 형식이 column 형태이다.
* 스칼라, 파이썬, 자바로 개발할 수 있으며, \
각 언어에 따라 성능은 달라질 수 있다.
* udf를 생성하면, 모든 워커 노드에서 사용할 수 있도록 \
스파크에 등록해야한다.
* 스파크는 드라이버에서 함수를 직렬화하고, 네트워크를 통해 익스큐터에게 전달한다.

### 스칼라 / 자바 udf
### 파이썬 udf
* 파이썬 udf의 경우, 스파크는 워커 노드에서 파이썬 프로세스를 실행한다.
* 파이썬이 이해할 수 있는 포맷으로 모든 데이터를 직렬화 한다.
* 파이썬 프로세스에 있는 데이터의 로우마다 함수를 실행하고, `JVM과 Spark`에 처리 결과를 반환합니다.
* 문제점
  - 파이썬 udf를 사용하면 데이터를 직렬화 하는 과정에서 \
  메모리에 사용에 제약이 생긴다.

In [51]:
from pyspark.sql.functions import udf

udfExampleDf = spark.range(5).toDF("num")

def power3(double_value):
    return double_value ** 3

power3(2.0)

power3udf = udf(power3)

In [52]:
udfExampleDf.select(power3udf(col("num"))).show(2)

+-----------+
|power3(num)|
+-----------+
|          0|
|          1|
+-----------+
only showing top 2 rows

